In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, auc, roc_curve



In [2]:
fd = pd.read_csv(r"C:\Users\virin\Desktop\Frauddetection\creditcard_2023.csv")   
# dataset from https://www.kaggle.com/datasets/nelgiriyewithana/credit-card-fraud-detection-dataset-2023?resource=download

In [3]:
fd.head()
#All the features are anonimised to "V" 1- 28, Other colums- ID, Amount, Class

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [4]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568630 entries, 0 to 568629
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      568630 non-null  int64  
 1   V1      568630 non-null  float64
 2   V2      568630 non-null  float64
 3   V3      568630 non-null  float64
 4   V4      568630 non-null  float64
 5   V5      568630 non-null  float64
 6   V6      568630 non-null  float64
 7   V7      568630 non-null  float64
 8   V8      568630 non-null  float64
 9   V9      568630 non-null  float64
 10  V10     568630 non-null  float64
 11  V11     568630 non-null  float64
 12  V12     568630 non-null  float64
 13  V13     568630 non-null  float64
 14  V14     568630 non-null  float64
 15  V15     568630 non-null  float64
 16  V16     568630 non-null  float64
 17  V17     568630 non-null  float64
 18  V18     568630 non-null  float64
 19  V19     568630 non-null  float64
 20  V20     568630 non-null  float64
 21  V21     56

In [5]:
fd.isnull().sum() 
# No nulls found
# If nulls existed:
# fd.fillna(fd.mean(), inplace=True) # If average value is used to fill nulls, other imputations Median, Mode, or Forward/backward fill (used in timeseries) or,
# ML based imputatiomns like regression, random forest, or other techniques like hotdeck/colddeck or random sampling

id        0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [6]:
# Dividing the dataset to x and y
x = fd.drop(["id",'Class'], axis = 1,errors= 'ignore') # Dropping ID as it is not needed, and Class as it is the value that we are predicting
y = fd['Class'] # As class is the value we are predicting

In [7]:
# To inspect columns included
# print(pd.DataFrame(x.columns)) # or
print(x.columns.tolist())

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']


In [8]:
# Split dataset into training and test sets (use valid variable names)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) 
# Splitting data helps in avoiding overfitting and also helps to give unbiased evaluation of the model
# 20% of the whole data is used for testing and 80% for training the model, random_state will help in reproducing 
# the exact results for the model

In [9]:
# Checking the shape
print(x_train.shape)
print(x_test.shape)

(454904, 29)
(113726, 29)


In [10]:
# Feature scaling :  Performed such that each feature will contribute equally to model's learning process, especially, when 
# features are on different scale (Example compare Amt columns = large value, V columns = much smaller values compared)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)